<a href="https://colab.research.google.com/github/David-Huson/CAP4601_Project2/blob/main/project02Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAP4601-Intro to AI Project 2
## Image Classification using Convolutional Neural Networks

### Install and Import Dependencies

In [ ]:
!pip install -q tensorflow tensorflow-datasets

In [ ]:
!pip install -U keras-tuner

In [ ]:
import matplotlib.pyplot as plt         # used to plot the graphs for the experiments
import numpy as np                      # used to format data into a matrix

import tensorflow as tf                 # used to evaluate and work with models
import tensorflow_datasets as tfds      # used to import datasets from tf

from tensorflow import keras            # used to build the network layers

In [ ]:
tfds.disable_progress_bar()             #keep the noteboook clean

### Prepare Data

In [ ]:
# Load the MNIST dataset.
builder = tfds.builder('mnist')
info = builder.info

ds_train = tfds.load(name="mnist", split="train")
ds_test = tfds.load(name="mnist", split="test")

### Show Examples

In [ ]:
fig = tfds.show_examples(ds_train, info)

### Data Formatting

In [ ]:
# Change sets to numpy arrays
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

In [ ]:
# Change element types to floats for keras
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255
test_images /= 255

train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

# Convert class vectors to binary class matrices
train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)
train_labels

### Train a Basic Convolutional Network

In [ ]:
# Setup our CNN topology
model = keras.Sequential([
  keras.layers.Conv2D(64, 3, activation='relu', input_shape=(28,28,1)),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, activation='softmax')
])

# Train our CNN
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_images, train_labels, batch_size=128, epochs=15)

In [ ]:
# Run our CNN over our test data.
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Accuracy: {accuracy*100:.2f}% ")

### A More Elaborate CNN

In [ ]:
# Setup the new CNN topology using 2 pooling layers
model = keras.Sequential([
  keras.layers.Conv2D(64, 3, activation='relu', input_shape=(28,28,1)),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dense(10, activation='softmax')
])

# Train our CNN
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_images, train_labels, batch_size=128, epochs=15)

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Accuracy: {accuracy*100:.2f}% ")

### Addressing overfitting with dropout

In [ ]:
# Setup the new CNN topology using 2 pooling layers
model = keras.Sequential([
  keras.layers.Conv2D(64, 3, activation='relu'),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Conv2D(32, 3, activation='relu'),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(1028, activation="relu"),
  keras.layers.Dense(10, activation='softmax')
])

# Train our CNN
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_images, train_labels, batch_size=128, epochs=15)

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Accuracy: {accuracy*100:.2f}% ")

### Hyperparameter tuning using Karas tuner

#### Experimening with the number of convolution layers and their size

In [ ]:
import keras_tuner

def build_model(hp):
  model = keras.Sequential()

  for i in range(hp.Int("Conv2D layers", min_value=0, max_value=3)):
    model.add(keras.layers.Conv2D(hp.Choice(f"layer {i} filters", [16, 32, 64, 128]), 3, activation="relu"))
    
  model.add(keras.layers.MaxPool2D(2,2))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(1028, activation="relu"))
  model.add(keras.layers.Dense(10, activation='softmax'))

  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  return model

tuner = keras_tuner.RandomSearch(build_model, objective='val_accuracy', max_trials=32)

tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs=15, batch_size=128)


In [ ]:
# tuner.results_summary()
best_model = tuner.get_best_models()[0]
best_model.evaluate(test_images, test_labels)
best_model.summary()

In [ ]:
class LineBuilder:
    def __init__(self, line):
        self.line = line
        self.xs = list(line.get_xdata())
        self.ys = list(line.get_ydata())
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        print('click', event)
        if event.inaxes!=self.line.axes: return
        self.xs.append(event.xdata)
        self.ys.append(event.ydata)
        self.line.set_data(self.xs, self.ys)
        self.line.figure.canvas.draw()

fig = plt.figure(figsize=(28,28))
ax = fig.add_subplot(111)
ax.set_title('click to add points')
line, = ax.plot([], [], linestyle="none", marker="o", color="r")
linebuilder = LineBuilder(line)

plt.show()
plt.savefig('./test.jpg')

# import imagio

# im = imageio.imread('./test.jpg')
# im_np = np.asarray(im)

# print(im_np.shape)

# best_model = tuner.get_best_models()[0]
# result = best_model.predict(im)